# Assignment 4

For this assignment, you will be writing P4 code which enables a switch to function as a joint router and firewall for an IPv6 network. The network topology is shown below.
<img src="./images/assignment4_topology.png" width="1000px">

Note: h23 will only be configured for a final test case, and begins unconfigured.

Authored by Alexander Wolosewicz, IIT, adapting some code by Nishanth Shyamkumar, IIT


## Setup

In [ ]:
# Your student ID / hawk username, for example "jdoe"
student_id = "cnynavarapu"
# To use verbose output 
# (prints most commands as they are run, for debugging help)
verbose = 0

In [ ]:
# Read configuration file
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
config_name = "cnynavarapu_assignment4.txt"
config_file = open(config_name, 'r')
configs = [line.rstrip('\n') for line in config_file]

#Config values
subnet1 = IPv6Network(configs[0])
subnet2 = IPv6Network(configs[1])
h11_ipv6 = IPv6Address(configs[2])
h12_ipv6 = IPv6Address(configs[3])
h13_ipv6 = IPv6Address(configs[4])
h21_ipv6 = IPv6Address(configs[5])
h22_ipv6 = IPv6Address(configs[6])
s1_macs = configs[7].split(',')
s2_macs = configs[8].split(',')
h11_mac = configs[9]
h12_mac = configs[10]
h13_mac = configs[11]
h21_mac = configs[12]
h22_mac = configs[13]
h23_mac = configs[14]
s1_ipv6 = IPv6Address(configs[15])
s2_ipv6 = IPv6Address(configs[16])
tcp_ports = configs[17].split(',')
udp_ports = configs[18].split(',')

## Create topology
The creation of the topology is done automatically for you

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice_name = "cnynavarapu_assignment4"
slice = fablib.new_slice(name=slice_name)
host_cores = 2
host_ram = 2
host_disk = 10
switch_cores = 4
switch_ram = 8
switch_disk = 10
image = "default_ubuntu_20"
#Build Subnets
try:
    #Subnet 1
    h11 = slice.add_node(name="h11", cores=host_cores, ram=host_ram, disk=host_disk, image=image)
    h11_iface = h11.add_component(model="NIC_Basic").get_interfaces()[0]
    h12 = slice.add_node(name="h12", cores=host_cores, ram=host_ram, disk=host_disk, image=image)
    h12_iface = h12.add_component(model="NIC_Basic").get_interfaces()[0]
    h13 = slice.add_node(name="h13", cores=host_cores, ram=host_ram, disk=host_disk, image=image)
    h13_iface = h13.add_component(model="NIC_Basic").get_interfaces()[0]
    s1 = slice.add_node(name="s1", cores=switch_cores, ram=switch_ram, disk=switch_disk, image=image)
    s1_iface0 = s1.add_component(model="NIC_Basic", name='s1p0').get_interfaces()[0]
    s1_iface1 = s1.add_component(model="NIC_Basic", name='s1p1').get_interfaces()[0]
    s1_iface2 = s1.add_component(model="NIC_Basic", name='s1p2').get_interfaces()[0]
    s1_iface3 = s1.add_component(model="NIC_Basic", name='s1p3').get_interfaces()[0]
    s1h11 = slice.add_l2network(name='s1h11', interfaces=[s1_iface1, h11_iface])
    s1h12 = slice.add_l2network(name='s1h12', interfaces=[s1_iface2, h12_iface])
    s1h13 = slice.add_l2network(name='s1h13', interfaces=[s1_iface3, h13_iface])
    #Subnet 2
    h21 = slice.add_node(name="h21", cores=host_cores, ram=host_ram, disk=host_disk, image=image)
    h21_iface = h21.add_component(model="NIC_Basic").get_interfaces()[0]
    h22 = slice.add_node(name="h22", cores=host_cores, ram=host_ram, disk=host_disk, image=image)
    h22_iface = h22.add_component(model="NIC_Basic").get_interfaces()[0]
    h23 = slice.add_node(name="h23", cores=host_cores, ram=host_ram, disk=host_disk, image=image)
    h23_iface = h23.add_component(model="NIC_Basic").get_interfaces()[0]
    s2 = slice.add_node(name="s2", cores=switch_cores, ram=switch_ram, disk=switch_disk, image=image)
    s2_iface0 = s2.add_component(model="NIC_Basic", name='s2p0').get_interfaces()[0]
    s2_iface1 = s2.add_component(model="NIC_Basic", name='s2p1').get_interfaces()[0]
    s2_iface2 = s2.add_component(model="NIC_Basic", name='s2p2').get_interfaces()[0]
    s2_iface3 = s2.add_component(model="NIC_Basic", name='s2p3').get_interfaces()[0]
    s2h21 = slice.add_l2network(name='s2h21', interfaces=[s2_iface1, h21_iface])
    s2h22 = slice.add_l2network(name='s2h22', interfaces=[s2_iface2, h22_iface])
    s2h23 = slice.add_l2network(name='s2h23', interfaces=[s2_iface3, h23_iface])
    s1s2 = slice.add_l2network(name='s1s2', interfaces=[s1_iface0, s2_iface0])
    slice.submit()
except Exception as e:
    print(f"Error: {e}")
s1 = slice.get_node(name='s1')
h11 = slice.get_node(name='h11')
h12 = slice.get_node(name='h12')
h13 = slice.get_node(name='h13')
s2 = slice.get_node(name='s2')
h21 = slice.get_node(name='h21')
h22 = slice.get_node(name='h22')
h23 = slice.get_node(name='h23')
switches = [s1, s2]
hosts = [h11, h12, h13, h21, h22, h23]

In [ ]:
# If the above had any node fail, UNCOMMENT the line below and execute this cell, then try again
#slice.delete()

In [ ]:
#Install scripts and turn up interfaces
try:
    install_bmv2 = '. /etc/os-release && echo "deb https://download.opensuse.org/repositories/home:/p4lang/xUbuntu_${VERSION_ID}/ /" | sudo tee /etc/apt/sources.list.d/home:p4lang.list && curl -L "https://download.opensuse.org/repositories/home:/p4lang/xUbuntu_${VERSION_ID}/Release.key" | sudo apt-key add - && sudo apt-get update && sudo apt install -y p4lang-p4c'
    install_net_tools = 'sudo apt-get install -y net-tools'
    install_scapy = 'sudo apt-get update && sudo apt-get install -y python3-scapy'
    for switch in switches:
        stdout, stderr = switch.execute(install_bmv2, quiet=True)
        stdout, stderr = switch.execute(install_net_tools, quiet=True)
        if type(ip_address(switch.get_management_ip())) is IPv6Address:
            switch.execute("sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
    for host in hosts:
        stdout, stderr = host.execute(f'{install_net_tools} && {install_scapy}', quiet=True)
        if type(ip_address(host.get_management_ip())) is IPv6Address:
            host.execute("sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
    #Turn up subnet 1 interfaces
    s1_iface0 = s1.get_interface(network_name='s1s2')
    s1_iface0_name = s1_iface0.get_device_name()
    stdout, stderr = s1.execute(f'sudo ip link set dev {s1_iface0_name} up', quiet=True)
    s1_iface1 = s1.get_interface(network_name='s1h11')
    s1_iface1_name = s1_iface1.get_device_name()
    stdout, stderr = s1.execute(f'sudo ip link set dev {s1_iface1_name} up', quiet=True)
    s1_iface2 = s1.get_interface(network_name='s1h12')
    s1_iface2_name = s1_iface2.get_device_name()
    stdout, stderr = s1.execute(f'sudo ip link set dev {s1_iface2_name} up', quiet=True)
    s1_iface3 = s1.get_interface(network_name='s1h13')
    s1_iface3_name = s1_iface3.get_device_name()
    stdout, stderr = s1.execute(f'sudo ip link set dev {s1_iface3_name} up', quiet=True)
    h11_iface = h11.get_interface(network_name='s1h11')
    h11_iface_name = h11_iface.get_device_name()
    stdout, stderr = h11.execute(f'sudo ip link set dev {h11_iface_name} up', quiet=True)
    h12_iface = h12.get_interface(network_name='s1h12')
    h12_iface_name = h12_iface.get_device_name()
    stdout, stderr = h12.execute(f'sudo ip link set dev {h12_iface_name} up', quiet=True)
    h13_iface = h13.get_interface(network_name='s1h13')
    h13_iface_name = h13_iface.get_device_name()
    stdout, stderr = h13.execute(f'sudo ip link set dev {h13_iface_name} up', quiet=True)
    #Turn up subnet 2 interfaces
    s2_iface0 = s2.get_interface(network_name='s1s2')
    s2_iface0_name = s2_iface0.get_device_name()
    stdout, stderr = s2.execute(f'sudo ip link set dev {s2_iface0_name} up', quiet=True)
    s2_iface1 = s2.get_interface(network_name='s2h21')
    s2_iface1_name = s2_iface1.get_device_name()
    stdout, stderr = s2.execute(f'sudo ip link set dev {s2_iface1_name} up', quiet=True)
    s2_iface2 = s2.get_interface(network_name='s2h22')
    s2_iface2_name = s2_iface2.get_device_name()
    stdout, stderr = s2.execute(f'sudo ip link set dev {s2_iface2_name} up', quiet=True)
    s2_iface3 = s2.get_interface(network_name='s2h23')
    s2_iface3_name = s2_iface3.get_device_name()
    stdout, stderr = s2.execute(f'sudo ip link set dev {s2_iface3_name} up', quiet=True)
    h21_iface = h21.get_interface(network_name='s2h21')
    h21_iface_name = h21_iface.get_device_name()
    stdout, stderr = h21.execute(f'sudo ip link set dev {h21_iface_name} up', quiet=True)
    h22_iface = h22.get_interface(network_name='s2h22')
    h22_iface_name = h22_iface.get_device_name()
    stdout, stderr = h22.execute(f'sudo ip link set dev {h22_iface_name} up', quiet=True)
    h23_iface = h23.get_interface(network_name='s2h23')
    h23_iface_name = h23_iface.get_device_name()
    stdout, stderr = h23.execute(f'sudo ip link set dev {h23_iface_name} up', quiet=True)
    #Enable IPv6 Forwarding
    stdout, stderr = s1.execute("sudo sysctl net.ipv6.conf.all.forwarding=1", quiet=True)
    stdout, stderr = s2.execute("sudo sysctl net.ipv6.conf.all.forwarding=1", quiet=True)
except Exception as e:
    print(f"Error: {e}")

In [ ]:
#Configure MAC and IP Addresses
try:
    #MACs
    if verbose: print(f's1: sudo ifconfig {s1_iface0_name} hw ether {s1_macs[0]}')
    stdout, stderr = s1.execute(f'sudo ifconfig {s1_iface0_name} hw ether {s1_macs[0]}')
    if verbose: print(f's1: sudo ifconfig {s1_iface1_name} hw ether {s1_macs[1]}')
    stdout, stderr = s1.execute(f'sudo ifconfig {s1_iface1_name} hw ether {s1_macs[1]}')
    if verbose: print(f's1: sudo ifconfig {s1_iface2_name} hw ether {s1_macs[2]}')
    stdout, stderr = s1.execute(f'sudo ifconfig {s1_iface2_name} hw ether {s1_macs[2]}')
    if verbose: print(f's1: sudo ifconfig {s1_iface3_name} hw ether {s1_macs[3]}')
    stdout, stderr = s1.execute(f'sudo ifconfig {s1_iface3_name} hw ether {s1_macs[3]}')
    if verbose: print(f'h11: sudo ifconfig {h11_iface_name} hw ether {h11_mac}')
    stdout, stderr = h11.execute(f'sudo ifconfig {h11_iface_name} hw ether {h11_mac}')
    if verbose: print(f'h12: sudo ifconfig {h12_iface_name} hw ether {h12_mac}')
    stdout, stderr = h12.execute(f'sudo ifconfig {h12_iface_name} hw ether {h12_mac}')
    if verbose: print(f'h13: sudo ifconfig {h13_iface_name} hw ether {h13_mac}')
    stdout, stderr = h13.execute(f'sudo ifconfig {h13_iface_name} hw ether {h13_mac}')
    if verbose: print(f's2: sudo ifconfig {s2_iface0_name} hw ether {s2_macs[0]}')
    stdout, stderr = s2.execute(f'sudo ifconfig {s2_iface0_name} hw ether {s2_macs[0]}')
    if verbose: print(f's2: sudo ifconfig {s2_iface1_name} hw ether {s2_macs[1]}')
    stdout, stderr = s2.execute(f'sudo ifconfig {s2_iface1_name} hw ether {s2_macs[1]}')
    if verbose: print(f's2: sudo ifconfig {s2_iface2_name} hw ether {s2_macs[2]}')
    stdout, stderr = s2.execute(f'sudo ifconfig {s2_iface2_name} hw ether {s2_macs[2]}')
    if verbose: print(f's2: sudo ifconfig {s2_iface3_name} hw ether {s2_macs[3]}')
    stdout, stderr = s2.execute(f'sudo ifconfig {s2_iface3_name} hw ether {s2_macs[3]}')
    if verbose: print(f'h21: sudo ifconfig {h21_iface_name} hw ether {h21_mac}')
    stdout, stderr = h21.execute(f'sudo ifconfig {h21_iface_name} hw ether {h21_mac}')
    if verbose: print(f'h22: sudo ifconfig {h22_iface_name} hw ether {h22_mac}')
    stdout, stderr = h22.execute(f'sudo ifconfig {h22_iface_name} hw ether {h22_mac}')
    if verbose: print(f'h23: sudo ifconfig {h23_iface_name} hw ether {h23_mac}')
    stdout, stderr = h23.execute(f'sudo ifconfig {h23_iface_name} hw ether {h23_mac}')
except Exception as e:
    print(f"Error: {e}")

In [ ]:
ips = {}
ips["h11"] = h11_ipv6
ips["h12"] = h12_ipv6
ips["h13"] = h13_ipv6
ips["h21"] = h21_ipv6
ips["h22"] = h22_ipv6
ifaces = {}
ifaces["h11"] = h11_iface_name
ifaces["h12"] = h12_iface_name
ifaces["h13"] = h13_iface_name
ifaces["h21"] = h21_iface_name
ifaces["h22"] = h22_iface_name
nhop_macs = {}
nhop_macs["h11"] = s1_macs[1]
nhop_macs["h12"] = s1_macs[2]
nhop_macs["h13"] = s1_macs[3]
nhop_macs["h21"] = s2_macs[1]
nhop_macs["h22"] = s2_macs[2]
switches = [s1, s2]
hosts = [h11, h12, h13, h21, h22, h23]
for host in hosts:
    hostname = host.get_name()
    if hostname == "h23":
        continue
    if verbose: print(f'{hostname}:sudo ifconfig {ifaces[hostname]} del {ips[hostname]} && sudo ip -6 route del {subnet2} && sudo ip -6 route del {subnet1}')
    stdout, stderr = host.execute(f'sudo ifconfig {ifaces[hostname]} del {ips[hostname]} && sudo ip -6 route del {subnet2} && sudo ip -6 route del {subnet1}')
    if verbose: print(f'{hostname}:sudo ifconfig {ifaces[hostname]} add {ips[hostname]} up && sudo ip -6 route add {subnet2} dev {ifaces[hostname]} && sudo ip -6 route add {subnet1} dev {ifaces[hostname]}')
    stdout, stderr = host.execute(f'sudo ifconfig {ifaces[hostname]} add {ips[hostname]} up && sudo ip -6 route add {subnet2} dev {ifaces[hostname]} && sudo ip -6 route add {subnet1} dev {ifaces[hostname]}')
    for dest in hosts:
        destname = dest.get_name()
        if dest == host or destname == "h23":
            continue
        if verbose: print(f'{hostname}:sudo ip -6 neigh del {ips[destname]} dev {ifaces[hostname]}')
        stdout, stderr = host.execute(f'sudo ip -6 neigh del {ips[destname]} dev {ifaces[hostname]}')
        if verbose: print(f'{hostname}:sudo ip -6 neigh add {ips[destname]} lladdr {nhop_macs[hostname]} dev {ifaces[hostname]}')
        stdout, stderr = host.execute(f'sudo ip -6 neigh add {ips[destname]} lladdr {nhop_macs[hostname]} dev {ifaces[hostname]}')

## Reset
If you ever close the Jupyter kernel, or it crashes, all variable values are lost. The below cell can be executed to reset all the variable values.
Note that if the slice expires (24 hours after creation) you must start from the beginning. Any progress made in files (the shell script or P4 code) is safe as long as it is saved.
Make sure to retype your student ID for "student_id".

In [ ]:
student_id = "cnynavarapu"
verbose = 0
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice_name = "cnynavarapu_assignment4"
slice = fablib.get_slice(name=slice_name)
s1 = slice.get_node(name='s1')
h11 = slice.get_node(name='h11')
h12 = slice.get_node(name='h12')
h13 = slice.get_node(name='h13')
s2 = slice.get_node(name='s2')
h21 = slice.get_node(name='h21')
h22 = slice.get_node(name='h22')
h23 = slice.get_node(name='h23')
s1_iface0_name = s1.get_interface(network_name='s1s2').get_device_name()
s1_iface1_name = s1.get_interface(network_name='s1h11').get_device_name()
s1_iface2_name = s1.get_interface(network_name='s1h12').get_device_name()
s1_iface3_name = s1.get_interface(network_name='s1h13').get_device_name()
h11_iface_name = h11.get_interface(network_name='s1h11').get_device_name()
h12_iface_name = h12.get_interface(network_name='s1h12').get_device_name()
h13_iface_name = h13.get_interface(network_name='s1h13').get_device_name()
s2_iface0_name = s2.get_interface(network_name='s1s2').get_device_name()
s2_iface1_name = s2.get_interface(network_name='s2h21').get_device_name()
s2_iface2_name = s2.get_interface(network_name='s2h22').get_device_name()
s2_iface3_name = s2.get_interface(network_name='s2h23').get_device_name()
h21_iface_name = h21.get_interface(network_name='s2h21').get_device_name()
h22_iface_name = h22.get_interface(network_name='s2h22').get_device_name()
h23_iface_name = h23.get_interface(network_name='s2h23').get_device_name()
switches = [s1, s2]
hosts = [h11, h12, h13, h21, h22, h23]
ips = {}
ips["h11"] = h11_ipv6
ips["h12"] = h12_ipv6
ips["h13"] = h13_ipv6
ips["h21"] = h21_ipv6
ips["h22"] = h22_ipv6
ifaces = {}
ifaces["h11"] = h11_iface_name
ifaces["h12"] = h12_iface_name
ifaces["h13"] = h13_iface_name
ifaces["h21"] = h21_iface_name
ifaces["h22"] = h22_iface_name
nhop_macs = {}
nhop_macs["h11"] = s1_macs[1]
nhop_macs["h12"] = s1_macs[2]
nhop_macs["h13"] = s1_macs[3]
nhop_macs["h21"] = s2_macs[1]
nhop_macs["h22"] = s2_macs[2]

# SSH Keys
The below cell will print the ssh keys, which you can enter into a new terminal tab to ssh to that client. Useful for testing/debugging your P4 code.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node.get_name()}: {node.get_ssh_command()}")
except Exception as e:
    print(f"Exception: {e}")

# P4 Programming
The main objective of this assignment is to create P4 code which will configure the switches (s1 and s2 in the topology) to function as IPv6 routers and firewalls. A copy of the P4 program from the tutorial, which created an IPv4 router, is provided as a starter template in [assignment4.p4](./assignment4.p4).

The routing requirements are that:
- hosts on each subnet can ping each other
- hosts can ping hosts on the other subnet, and the switches route to the other subnet using LPM

The firewall requirement is that TCP or UDP packets must have one of the ports listed below as either the source or destination port.

Besides modifying P4 code, you will also need to edit [add_rules_s1.sh](./add_rules_s1.sh) and [add_rules_s2.sh](./add_rules_s2.sh) to contain the control plane commands the switches need to be configured with. This has the additional benefit of making testing easier, since you can ssh into the switch and start the bmv2 process using run_bmv2 scripts, then ssh into the switch again in a second terminal and execute your add_rules script to add all the rules.

In [ ]:
print(f"s1's MAC addresses are (in order from port 0 to port 3):\n{s1_macs[0]}\n{s1_macs[1]}\n{s1_macs[2]}\n{s1_macs[3]}")
print(f"s2's MAC addresses are (in order from port 0 to port 3):\n{s2_macs[0]}\n{s2_macs[1]}\n{s2_macs[2]}\n{s2_macs[3]}")
print("For the above, s1 and s2 are connected to each other on their port 0. Port 1 connects to h11 or h21, port 2 to h12 or h22, and port 3 to h13 or h23.")
print(f"h11's MAC address is {h11_mac} and its IPv6 address is {h11_ipv6}")
print(f"h12's MAC address is {h12_mac} and its IPv6 address is {h12_ipv6}")
print(f"h13's MAC address is {h13_mac} and its IPv6 address is {h13_ipv6}")
print(f"h21's MAC address is {h21_mac} and its IPv6 address is {h21_ipv6}")
print(f"h22's MAC address is {h22_mac} and its IPv6 address is {h22_ipv6}")
print(f"The allowed TCP ports are:")
for port in tcp_ports:
    print(port)
print(f"The allowed UDP ports are:")
for port in udp_ports:
    print(port)
print("A reminder that in the topology, h23 is ignored and autoconfigured later.")

In [ ]:
# Create BMv2 helper scripts - this cell only needs to run once
s_ifaces = {}
s_ifaces["s1"] = [s1_iface0_name, s1_iface1_name, s1_iface2_name, s1_iface3_name]
s_ifaces["s2"] = [s2_iface0_name, s2_iface1_name, s2_iface2_name, s2_iface3_name]
for switch in switches:
    switchname = switch.get_name()
    outfile = "run_bmv2_" + switchname + ".sh"
    with open(outfile, 'w') as out:
        out.write("#!/bin/bash\n\n")
        out.write("p4c --target bmv2 --arch v1model --std p4-14 ~/assignment4.p4 -o ~\n\n")
        out.write(f"sudo simple_switch -i 0@{s_ifaces[switchname][0]} -i 1@{s_ifaces[switchname][1]} -i 2@{s_ifaces[switchname][2]} -i 3@{s_ifaces[switchname][3]} ~/assignment4.json --log-console")
        out.close()

In [ ]:
# Build mass upload file - this cell only needs to run once
try:
    scps = []
    outfile = "uploader.sh"
    for switch in switches:
        switchname = switch.get_name()
        ssh = slice.get_node(name=switchname).get_ssh_command()
        param = ssh.split("ssh ")[1]
        scp = "scp " + param
        scp_addr = '[' + scp.split('@')[1] + ']'
        scp_config = scp.split('@')[0]
        scp_base = scp_config + '@' + scp_addr
        scp_bmv2 = scp_base.split("ubuntu")[0] + "run_bmv2_" + switchname + ".sh ubuntu" + scp_base.split("ubuntu")[1] + ":~"
        scp_rules = scp_base.split("ubuntu")[0] + "add_rules_" + switchname + ".sh ubuntu" + scp_base.split("ubuntu")[1] + ":~"
        scp_p4 = scp_base.split("ubuntu")[0] + "assignment4.p4 ubuntu" + scp_base.split("ubuntu")[1] + ":~"
        scps.append(scp_bmv2)
        scps.append(scp_rules)
        scps.append(scp_p4)
        if switchname == "s2":
            scp_tester = scp_base.split("ubuntu")[0] + "tester.json ubuntu" + scp_base.split("ubuntu")[1] + ":~"
            scps.append(scp_tester)
            scp_tester_rules = scp_base.split("ubuntu")[0] + "tester_rules_s2.sh ubuntu" + scp_base.split("ubuntu")[1] + ":~"
            scps.append(scp_tester_rules)
    for host in hosts:
        hostname = host.get_name()
        ssh = slice.get_node(name=hostname).get_ssh_command()
        param = ssh.split("ssh ")[1]
        scp = "scp " + param
        scp_addr = '[' + scp.split('@')[1] + ']'
        scp_config = scp.split('@')[0]
        scp_base = scp_config + '@' + scp_addr
        scp_send = scp_base.split("ubuntu")[0] + "send.py ubuntu" + scp_base.split("ubuntu")[1] + ":~"
        scp_receive = scp_base.split("ubuntu")[0] + "receive.py ubuntu" + scp_base.split("ubuntu")[1] + ":~"
        scps.append(scp_send)
        scps.append(scp_receive)
    with open(outfile, 'w') as out:
        out.write("#!/bin/bash\n\n")
        for command in scps:
            out.write(command + "\n")
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
%%sh
# Uploader - initial upload, only needed once
chmod u+x uploader.sh
./uploader.sh

In [ ]:
outfile = "uploader2.sh"
with open(outfile, 'w') as out:
    out.write("#!/bin/bash\n\n")
    for command in scps[:6]:
        out.write(command + '\n')
stdout, stderr = s1.execute("chmod u+x run_bmv2_s1.sh && chmod u+x add_rules_s1.sh", quiet=True)
stdout, stderr = s2.execute("chmod u+x run_bmv2_s2.sh && chmod u+x add_rules_s2.sh", quiet=True)

In [ ]:
%%sh
# Uploader2 - pushes changes for switch files, execute as needed
chmod u+x uploader2.sh
./uploader2.sh

# Testing
The below cells will automatically test your P4 and sh scripts. The score breakdown for this assignment is:
- 2.5pts for h11 through h21 being able to ping each other
- 5 pts for the switches correctly filtering TCP and UDP ports
- 2.5pts for correctly routing to a random host added to subnet 2

There is a total of 10 points. These cells should not be modified. Additionally, the cells are reliant on the previous cells, so they must be executed in order.

In [ ]:
# Ping Test
from time import sleep
pingscore = 0
stdout, stderr = s1.execute("sudo killall simple_switch", quiet=True)
stdout, stderr = s2.execute("sudo killall simple_switch", quiet=True)
s1.execute_thread("./run_bmv2_s1.sh")
s2.execute_thread("./run_bmv2_s2.sh")
sleep(5)
s1.execute("./add_rules_s1.sh", quiet=True)
s2.execute("./add_rules_s2.sh", quiet=True)
for host in hosts:
    hostname = host.get_name()
    counter = 0
    if hostname == "h23": continue
    for dest in hosts:
        destname = dest.get_name()
        if dest == host or destname == "h23": continue
        if host.ping_test(ips[destname]):
            counter += 1
        else:
            print(f"{hostname} failed to ping {destname}")
    if counter == 4:
        print(f"{hostname} 4/4 pings successful, +0.5pts")
        pingscore += 0.5
    else:
        print(f"{hostname} {counter}/4 pings successful, 0pts")
print(f"Total ping score: {pingscore}/2.5pts")

In [ ]:
# Port Test
import random
portscore = 0
random.seed(a=student_id)
tcp_good = random.choice(tcp_ports)
tcp_bad = random.randint(1, 65535)
udp_good = random.choice(udp_ports)
udp_bad = random.randint(1, 65535)
while tcp_bad in tcp_ports:
    tcp_bad = random.randint(1, 65535)
while udp_bad in udp_ports:
    udp_bad = random.randint(1, 65535)
sender = random.choice(hosts[:3])
sender_name = sender.get_name()
receiver = random.choice(hosts[3:5])
receiver_name = receiver.get_name()
stdout, stderr = sender.execute("chmod u+x send.py", quiet=True)
stdout, stderr = sender.execute("chmod u+x receive.py", quiet=True)
stdout, stderr = receiver.execute("chmod u+x send.py", quiet=True)
stdout, stderr = receiver.execute("chmod u+x receive.py", quiet=True)
print(f"{receiver_name}: Starting receiver")
rx_thread = receiver.execute_thread(f"sudo ./receive.py --host_iface {ifaces[receiver_name]}")
sleep(5)
if verbose: print(f"TCP good: {tcp_good}, TCP bad: {tcp_bad}, UDP good: {udp_good}, UDP bad: {udp_bad}")
print(f"{sender_name}: Send allowed TCP packets")
if verbose: print(f"sudo ./send.py {ips[receiver_name]} tcp {tcp_good} {tcp_bad} --host_iface {ifaces[sender_name]}")
stdout, stderr = sender.execute(f"sudo ./send.py {ips[receiver_name]} tcp {tcp_good} {tcp_bad} --host_iface {ifaces[sender_name]}", quiet=True)
sleep(0.1)
if verbose: print(f"sudo ./send.py {ips[receiver_name]} tcp {tcp_bad} {tcp_good} --host_iface {ifaces[sender_name]}")
stdout, stderr = sender.execute(f"sudo ./send.py {ips[receiver_name]} tcp {tcp_bad} {tcp_good} --host_iface {ifaces[sender_name]}", quiet=True)
sleep(0.1)
print(f"{sender_name}: Send non-allowed TCP packet")
if verbose: print(f"sudo ./send.py {ips[receiver_name]} tcp {tcp_bad} {tcp_bad} --host_iface {ifaces[sender_name]}")
stdout, stderr = sender.execute(f"sudo ./send.py {ips[receiver_name]} tcp {tcp_bad} {tcp_bad} --host_iface {ifaces[sender_name]}", quiet=True)
sleep(0.1)
print(f"{sender_name}: Send allowed UDP packets")
if verbose: print(f"sudo ./send.py {ips[receiver_name]} udp {udp_good} {udp_bad} --host_iface {ifaces[sender_name]}")
stdout, stderr = sender.execute(f"sudo ./send.py {ips[receiver_name]} udp {udp_good} {udp_bad} --host_iface {ifaces[sender_name]}", quiet=True)
sleep(0.1)
if verbose: print(f"sudo ./send.py {ips[receiver_name]} udp {udp_bad} {udp_good} --host_iface {ifaces[sender_name]}")
stdout, stderr = sender.execute(f"sudo ./send.py {ips[receiver_name]} udp {udp_bad} {udp_good} --host_iface {ifaces[sender_name]}", quiet=True)
sleep(0.1)
print(f"{sender_name}: Send non-allowed UDP packet")
if verbose: print(f"sudo ./send.py {ips[receiver_name]} udp {udp_bad} {udp_bad} --host_iface {ifaces[sender_name]}")
stdout, stderr = sender.execute(f"sudo ./send.py {ips[receiver_name]} udp {udp_bad} {udp_bad} --host_iface {ifaces[sender_name]}", quiet=True)
sleep(0.1)
stdout, stderr = receiver.execute("sudo killall python3", quiet=True)
rx_results = rx_thread.result()[0].split('\n')
if verbose: print(rx_results)
count_tcp = 0
count_udp = 0
for result in rx_results[1:]:
    if result == "Received TCP":
        count_tcp += 1
    elif result == "Received UDP":
        count_udp += 1
if count_tcp == 1:
    print("Received 1 TCP packet, should be 2. +0pts")
elif count_tcp == 2:
    portscore += 2.5
    print("Received 2 TCP packets. 2.5pts")
elif count_tcp == 3:
    print("Received 3 TCP packets, should be 2. +0pts")
if count_udp == 1:
    print("Received 1 UDP packet, should be 2. +0pts")
elif count_udp == 2:
    portscore += 2.5
    print("Received 2 UDP packets. 2.5pts")
elif count_udp == 3:
    print("Received 3 UDP packets, should be 2. +0pts")
print(f"Total port score: {portscore}/5.0pts")

In [ ]:
# h23 Test
stdout, stderr = s2.execute("sudo killall simple_switch", quiet=True)
s2.execute_thread(f"sudo simple_switch -i 0@{s2_iface0_name} -i 1@{s2_iface1_name} -i 2@{s2_iface2_name} -i 3@{s2_iface3_name} ~/tester.json --log-console")
sleep(5)
hexchars = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', 'a', 'b', 'c', 'd', 'e', 'f']
try:
    h23_ipv6_old = h23_ipv6
    h23_ipv6 = "2001:db8:1812:abc2::"+random.choice(hexchars)+random.choice(hexchars)+random.choice(hexchars)+random.choice(hexchars)
except:
    h23_ipv6 = "2001:db8:1812:abc2::"+random.choice(hexchars)+random.choice(hexchars)+random.choice(hexchars)+random.choice(hexchars)
    h23_ipv6_old = h23_ipv6
if verbose: print(f"chmod u+x tester_rules_s2.sh && ./tester_rules_s2.sh {s2_macs[0]} {s2_macs[1]} {s2_macs[2]} {s2_macs[3]} {h21_mac} {h22_mac} {h23_mac} {h21_ipv6} {h22_ipv6} {h23_ipv6} {s1_ipv6} {s1_macs[0]} {subnet1}")
s2.execute(f"chmod u+x tester_rules_s2.sh && ./tester_rules_s2.sh {s2_macs[0]} {s2_macs[1]} {s2_macs[2]} {s2_macs[3]} {h21_mac} {h22_mac} {h23_mac} {h21_ipv6} {h22_ipv6} {h23_ipv6} {s1_ipv6} {s1_macs[0]} {subnet1}", quiet=True)

if verbose: print(f'sudo ifconfig {h23_iface_name} del {h23_ipv6_old} && sudo ip -6 route del {subnet2} && sudo ip -6 route del {subnet1}')
stdout, stderr = h23.execute(f'sudo ifconfig {h23_iface_name} del {h23_ipv6_old} && sudo ip -6 route del {subnet2} && sudo ip -6 route del {subnet1}')
if verbose: print(f'sudo ifconfig {h23_iface_name} add {h23_ipv6} up && sudo ip -6 route add {subnet2} dev {h23_iface_name} && sudo ip -6 route add {subnet1} dev {h23_iface_name}')
stdout, stderr = h23.execute(f'sudo ifconfig {h23_iface_name} add {h23_ipv6} up && sudo ip -6 route add {subnet2} dev {h23_iface_name} && sudo ip -6 route add {subnet1} dev {h23_iface_name}')
for dest in hosts:
    destname = dest.get_name()
    if dest == host:
        continue
    if verbose: print(f'h23:sudo ip -6 neigh del {ips[destname]} dev {h23_iface_name}')
    stdout, stderr = h23.execute(f'sudo ip -6 neigh del {ips[destname]} dev {h23_iface_name}')
    if verbose: print(f'h23:sudo ip -6 neigh add {ips[destname]} lladdr {s2_macs[3]} dev {h23_iface_name}')
    stdout, stderr = h23.execute(f'sudo ip -6 neigh add {ips[destname]} lladdr {s2_macs[3]} dev {h23_iface_name}')
    if verbose: print(f'{destname}:sudo ip -6 neigh del {h23_ipv6_old} dev {ifaces[destname]}')
    stdout, stderr = dest.execute(f'sudo ip -6 neigh del {h23_ipv6_old} dev {ifaces[destname]}')
    if verbose: print(f'{destname}:sudo ip -6 neigh add {h23_ipv6} lladdr {nhop_macs[destname]} dev {ifaces[destname]}')
    stdout, stderr = dest.execute(f'sudo ip -6 neigh add {h23_ipv6} lladdr {nhop_macs[destname]} dev {ifaces[destname]}')
counter = 0
for dest in hosts:
    destname = dest.get_name()
    if destname == "h23": continue
    if h23.ping_test(ips[destname]):
        counter += 1
    else:
        print(f"h23 failed to ping {destname}")
h23score = 0
if counter == 5:
    print(f"h23 5/5 pings successful, +2.5pts")
    h23score = 2.5
else:
    print(f"h23 {counter}/5 pings successful, 0pts")
print(f"Total Score: {pingscore+portscore+h23score}/10pts")

In [ ]:
slice.delete()